In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath("../../"))

In [2]:
from brmspy.modulesession.module_session import RModuleSession
_module_path = "brmspy.brms"

In [ ]:
from types import ModuleType
from typing import cast

import brmspy.brms as _brms_mod

brms = cast(
    ModuleType,
    RModuleSession(
        module=_brms_mod,
        module_path=_module_path,
        runtime_conf={
            # e.g. "r_home": "...", "startup": [...]
        },
    ),
)

In [10]:
brms.install_brms(use_prebuilt=True)

cmd CALL


R callback write-console: Error in loadNamespace(x) : there is no package called ‘cmdstanr’
  
R callback write-console: CmdStan path set to: /Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0/cmdstan
  


PosixPath('/Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0')

In [11]:
data = brms.get_brms_data("epilepsy")

cmd CALL


In [12]:
formula = "count ~ zAge + zBase * Trt + (1|patient)"
family = "poisson"
result = brms.brm(
    formula=formula,
    data=data,
    family=family,
    warmup=500,
    iter=1000,
    chains=4
)


cmd CALL


[brmspy][worker_main] Fitting model with brms (backend: cmdstanr)...
R callback write-console: Compiling Stan program...
  


R callback write-console: Start sampling
  


Running MCMC with 4 chains, at most 2 in parallel...

Chain 1 Iteration:   1 / 1000 [  0%]  (Warmup) 
Chain 2 Iteration:   1 / 1000 [  0%]  (Warmup) 
Chain 1 Iteration: 100 / 1000 [ 10%]  (Warmup) 
Chain 2 Iteration: 100 / 1000 [ 10%]  (Warmup) 
Chain 1 Iteration: 200 / 1000 [ 20%]  (Warmup) 
Chain 2 Iteration: 200 / 1000 [ 20%]  (Warmup) 
Chain 1 Iteration: 300 / 1000 [ 30%]  (Warmup) 
Chain 2 Iteration: 300 / 1000 [ 30%]  (Warmup) 
Chain 1 Iteration: 400 / 1000 [ 40%]  (Warmup) 
Chain 2 Iteration: 400 / 1000 [ 40%]  (Warmup) 
Chain 1 Iteration: 500 / 1000 [ 50%]  (Warmup) 
Chain 1 Iteration: 501 / 1000 [ 50%]  (Sampling) 
Chain 2 Iteration: 500 / 1000 [ 50%]  (Warmup) 
Chain 2 Iteration: 501 / 1000 [ 50%]  (Sampling) 
Chain 1 Iteration: 600 / 1000 [ 60%]  (Sampling) 
Chain 2 Iteration: 600 / 1000 [ 60%]  (Sampling) 
Chain 1 Iteration: 700 / 1000 [ 70%]  (Sampling) 
Chain 2 Iteration: 700 / 1000 [ 70%]  (Sampling) 
Chain 1 Iteration: 800 / 1000 [ 80%]  (Sampling) 
Chain 2 Iteration: 8

In [16]:
brms.shutdown()